In [5]:
! rm -rf nano-BERT
! rm -rf space-model
! git clone https://github.com/StepanTita/nano-BERT.git
! git clone https://github.com/StepanTita/space-model.git

Cloning into 'nano-BERT'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 32 (delta 13), reused 22 (delta 6), pack-reused 0
Receiving objects: 100% (32/32), 38.28 MiB | 27.78 MiB/s, done.
Resolving deltas: 100% (13/13), done.
Cloning into 'space-model'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 19 (delta 4), reused 18 (delta 3), pack-reused 0
Receiving objects: 100% (19/19), 2.08 MiB | 13.81 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [6]:
import sys

sys.path.append('/content/nano-BERT')
sys.path.append('/content/space-model')

In [23]:
import math
import json
from collections import Counter

import torch
import torch.nn.functional as F

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

from tqdm import tqdm

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from model import NanoBertForClassification
from tokenizer import WordTokenizer

from space_model.model import SpaceModelForClassification
from space_model.loss import *

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
data = None
with open('nano-BERT/data/imdb_train.json') as f:
    data = [json.loads(l) for l in f.readlines()]

In [13]:
vocab = set()
for d in data:
    vocab |= set([w.lower() for w in d['text']])

In [14]:
test_data = None
with open('nano-BERT/data/imdb_test.json') as f:
   test_data = [json.loads(l) for l in f.readlines()]

In [15]:
def encode_label(label):
    if label == 'pos':
        return 1
    elif label == 'neg':
        return 0
    raise Exception(f'Unknown Label: {label}!')


class IMDBDataloader:
    def __init__(self, data, test_data, tokenizer, label_encoder, batch_size, val_frac=0.2):
        train_data, val_data = train_test_split(data, shuffle=True, random_state=42, test_size=val_frac)

        self.splits = {
            'train': [d['text'] for d in train_data],
            'test': [d['text'] for d in test_data],
            'val': [d['text'] for d in val_data]
        }

        self.labels = {
            'train': [d['label'] for d in train_data],
            'test': [d['label'] for d in test_data],
            'val': [d['label'] for d in val_data]
        }

        self.tokenized = {
            'train': [tokenizer(record).unsqueeze(0) for record in
                      tqdm(self.splits['train'], desc='Train Tokenization')],
            'test': [tokenizer(record).unsqueeze(0) for record in tqdm(self.splits['test'], desc='Test Tokenization')],
            'val': [tokenizer(record).unsqueeze(0) for record in tqdm(self.splits['val'], desc='Val Tokenization')],
        }

        self.encoded_labels = {
            'train': [label_encoder(label) for label in tqdm(self.labels['train'], desc='Train Label Encoding')],
            'test': [label_encoder(label) for label in tqdm(self.labels['test'], desc='Test Label Encoding')],
            'val': [label_encoder(label) for label in tqdm(self.labels['val'], desc='Val Label Encoding')],
        }

        self.curr_batch = 0
        self.batch_size = batch_size
        self.iterate_split = None

    def peek(self, split):
        return {
            'input_ids': self.splits[split][self.batch_size * self.curr_batch:self.batch_size * (self.curr_batch + 1)],
            'label_ids': self.labels[split][self.batch_size * self.curr_batch:self.batch_size * (self.curr_batch + 1)],
        }

    def take(self, split):
        batch = self.splits[split][self.batch_size * self.curr_batch:self.batch_size * (self.curr_batch + 1)]
        labels = self.labels[split][self.batch_size * self.curr_batch:self.batch_size * (self.curr_batch + 1)]
        self.curr_batch += 1
        return {
            'input_ids': batch,
            'label_ids': labels,
        }

    def peek_tokenized(self, split):
        return {
            'input_ids': torch.cat(
                self.tokenized[split][self.batch_size * self.curr_batch:self.batch_size * (self.curr_batch + 1)],
                dim=0),
            'label_ids': torch.tensor(
                self.encoded_labels[split][self.batch_size * self.curr_batch:self.batch_size * (self.curr_batch + 1)],
                dtype=torch.long),
        }

    def peek_index_tokenized(self, index, split):
        return {
            'input_ids': torch.cat(
                [self.tokenized[split][index]],
                dim=0),
            'label_ids': torch.tensor(
                [self.encoded_labels[split][index]],
                dtype=torch.long),
        }

    def peek_index(self, index, split):
        return {
            'input_ids': [self.splits[split][index]],
            'label_ids': [self.labels[split][index]],
        }

    def take_tokenized(self, split):
        batch = self.tokenized[split][self.batch_size * self.curr_batch:self.batch_size * (self.curr_batch + 1)]
        labels = self.encoded_labels[split][self.batch_size * self.curr_batch:self.batch_size * (self.curr_batch + 1)]
        self.curr_batch += 1
        return {
            'input_ids': torch.cat(batch, dim=0),
            'label_ids': torch.tensor(labels, dtype=torch.long),
        }

    def get_split(self, split):
        self.iterate_split = split
        return self

    def steps(self, split):
        return len(self.tokenized[split]) // self.batch_size

    def __iter__(self):
        self.reset()
        return self

    def __next__(self):
        if self.batch_size * self.curr_batch < len(self.splits[self.iterate_split]):
            return self.take_tokenized(self.iterate_split)
        else:
            raise StopIteration

    def reset(self):
        self.curr_batch = 0

In [16]:
NUM_EPOCHS = 50
BATCH_SIZE = 32
MAX_SEQ_LEN = 128
LEARNING_RATE = 1e-2

In [17]:
tokenizer = WordTokenizer(vocab=vocab, max_seq_len=MAX_SEQ_LEN)
tokenizer

Tokenizer[vocab=101522,self.special_tokens=['[PAD]', '[CLS]', '[SEP]', '[UNK]'],self.sep=' ',self.max_seq_len=128]

In [18]:
dataloader = IMDBDataloader(data, test_data, tokenizer, encode_label, batch_size=BATCH_SIZE)

Val Label Encoding: 100%|██████████| 5000/5000 [00:00<00:00, 1350387.64it/s]


In [19]:
bert = NanoBertForClassification(
    vocab_size=len(tokenizer.vocab),
    n_layers=1,
    n_heads=1,
    max_seq_len=MAX_SEQ_LEN,
    n_classes=2
).to(device)
bert

NanoBertForClassification(
  (nano_bert): NanoBERT(
    (embedding): BertEmbeddings(
      (word_embeddings): Embedding(101522, 3)
      (pos_embeddings): Embedding(128, 3)
      (layer_norm): LayerNorm((3,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layers): ModuleList(
        (0): BertLayer(
          (layer_norm1): LayerNorm((3,), eps=1e-05, elementwise_affine=True)
          (self_attention): BertSelfAttention(
            (heads): ModuleList(
              (0): BertAttentionHead(
                (query): Linear(in_features=3, out_features=3, bias=True)
                (key): Linear(in_features=3, out_features=3, bias=True)
                (values): Linear(in_features=3, out_features=3, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
            )
            (proj): Linear(in_features=3, out_features=3, bias=True)
            (dropout): Dropout(p=0.1, inplace=Fals

In [20]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [21]:
count_parameters(bert)

305123

In [22]:
optimizer = torch.optim.Adam(bert.parameters(), lr=LEARNING_RATE)

for i in range(NUM_EPOCHS):
    print(f'Epoch: {i + 1}')
    train_loss = 0.0
    train_preds = []
    train_labels = []

    bert.train()
    for step, batch in enumerate(tqdm(dataloader.get_split('train'), total=dataloader.steps('train'))):
        logits = bert(batch['input_ids'].to(device)) # (B, Seq_Len, 2)

        probs = F.softmax(logits[:, 0, :], dim=-1).cpu()
        pred = torch.argmax(probs, dim=-1) # (B)
        train_preds += pred.detach().tolist()
        train_labels += [l.item() for l in batch['label_ids']]

        loss = F.cross_entropy(logits[:, 0, :].cpu(), batch['label_ids'])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    val_loss = 0.0
    val_preds = []
    val_labels = []

    bert.eval()
    for step, batch in enumerate(tqdm(dataloader.get_split('val'), total=dataloader.steps('val'))):
        logits = bert(batch['input_ids'].to(device))

        probs = F.softmax(logits[:, 0, :], dim=-1).cpu()
        pred = torch.argmax(probs, dim=-1) # (B)
        val_preds += pred.detach().tolist()
        val_labels += [l.item() for l in batch['label_ids']]

        loss = F.cross_entropy(logits[:, 0, :].cpu(), batch['label_ids'])

        val_loss += loss.item()

    print()
    print(f'Train loss: {train_loss / dataloader.steps("train")} | Val loss: {val_loss / dataloader.steps("val")}')
    print(f'Train acc: {accuracy_score(train_labels, train_preds)} | Val acc: {accuracy_score(val_labels, val_preds)}')
    print(f'Train f1: {f1_score(train_labels, train_preds)} | Val f1: {f1_score(val_labels, val_preds)}')

Epoch: 1


100%|██████████| 625/625 [00:05<00:00, 110.48it/s]
157it [00:00, 174.67it/s]                         



Train loss: 0.69424509973526 | Val loss: 0.6982253128901507
Train acc: 0.5035 | Val acc: 0.5034
Train f1: 0.44251066696609026 | Val f1: 0.5033006601320265
Epoch: 2


100%|██████████| 625/625 [00:03<00:00, 201.68it/s]
157it [00:00, 242.28it/s]                         



Train loss: 0.6143448389053344 | Val loss: 0.5143855911416885
Train acc: 0.6317 | Val acc: 0.7508
Train f1: 0.6353465346534652 | Val f1: 0.7579642579642579
Epoch: 3


...


Train loss: 0.06323236619406379 | Val loss: 0.009533259160656225
Train acc: 0.9772 | Val acc: 0.9982
Train f1: 0.9772227772227773 | Val f1: 0.9982118021060997
Epoch: 49


100%|██████████| 625/625 [00:03<00:00, 185.82it/s]
157it [00:00, 235.86it/s]                         



Train loss: 0.06699056839924306 | Val loss: 0.009622795951005313
Train acc: 0.9772 | Val acc: 0.9986
Train f1: 0.9772227772227773 | Val f1: 0.9986080731755816
Epoch: 50


100%|██████████| 625/625 [00:02<00:00, 227.72it/s]
157it [00:00, 240.14it/s]                         


Train loss: 0.06226340431179851 | Val loss: 0.007051446901804854
Train acc: 0.97735 | Val acc: 0.9994
Train f1: 0.9774008480917935 | Val f1: 0.999403459932392


In [24]:
test_loss = 0.0
test_preds = []
test_labels = []

bert.eval()
for step, batch in enumerate(tqdm(dataloader.get_split('test'), total=dataloader.steps('test'))):
    logits = bert(batch['input_ids'].to(device))

    probs = F.softmax(logits[:, 0, :], dim=-1).cpu()
    pred = torch.argmax(probs, dim=-1) # (B)
    test_preds += pred.detach().tolist()
    test_labels += [l.item() for l in batch['label_ids']]

    loss = F.cross_entropy(logits[:, 0, :].cpu(), batch['label_ids'])

    test_loss += loss.item()

print()
print(f'Test loss: {test_loss / dataloader.steps("test")}')
print(f'Test acc: {accuracy_score(test_labels, test_preds)}')
print(f'Test f1: {f1_score(test_labels, test_preds)}')

782it [00:03, 225.12it/s]                         



Test loss: 0.0819332813621452
Test acc: 0.98692
Test f1: 0.987061290705496


# Add Space Model

In [96]:
class SpaceBertForClassification(torch.nn.Module):
    def __init__(self, base_model, n_embed=3, n_latent=3, n_concept_spaces=2, fine_tune=True):
        super().__init__()

        if fine_tune:
            for p in base_model.parameters():
                p.requires_grad_(False)
        self.bert = base_model
        self.space_model = SpaceModelForClassification(n_embed, n_latent, n_concept_spaces)

    def forward(self, x):
        embed = self.bert(x)

        out = self.space_model(embed)

        return out

In [97]:
space_bert = SpaceBertForClassification(bert.nano_bert).to(device)
space_bert

SpaceBertForClassification(
  (bert): NanoBERT(
    (embedding): BertEmbeddings(
      (word_embeddings): Embedding(101522, 3)
      (pos_embeddings): Embedding(128, 3)
      (layer_norm): LayerNorm((3,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layers): ModuleList(
        (0): BertLayer(
          (layer_norm1): LayerNorm((3,), eps=1e-05, elementwise_affine=True)
          (self_attention): BertSelfAttention(
            (heads): ModuleList(
              (0): BertAttentionHead(
                (query): Linear(in_features=3, out_features=3, bias=True)
                (key): Linear(in_features=3, out_features=3, bias=True)
                (values): Linear(in_features=3, out_features=3, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
            )
            (proj): Linear(in_features=3, out_features=3, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
 

In [98]:
count_parameters(space_bert)

32

In [100]:
NUM_OPTIM_EPOCHS = 5
LEARNING_RATE = 1e-2

l1 = 1e-6
l2 = 1e-6

In [101]:
optimizer = torch.optim.Adam(space_bert.parameters(), lr=LEARNING_RATE)

for i in range(NUM_OPTIM_EPOCHS):
    print(f'Epoch: {i + 1}')
    train_loss = 0.0
    train_preds = []
    train_labels = []

    bert.train()
    for step, batch in enumerate(tqdm(dataloader.get_split('train'), total=dataloader.steps('train'))):
        out = space_bert(batch['input_ids'].to(device)) # (B, 2)

        logits = out.logits.cpu()
        concept_spaces = [c.cpu() for c in out.concept_spaces]

        probs = F.softmax(logits, dim=-1).cpu()
        pred = torch.argmax(probs, dim=-1) # (B)
        train_preds += pred.detach().tolist()
        train_labels += [l.item() for l in batch['label_ids']]

        # CE + l1 * inter_loss + l2 * intra_loss
        loss = F.cross_entropy(logits, batch['label_ids']) + l1 * inter_space_loss(concept_spaces) + l2 * intra_space_loss(concept_spaces)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    val_loss = 0.0
    val_preds = []
    val_labels = []

    bert.eval()
    for step, batch in enumerate(tqdm(dataloader.get_split('val'), total=dataloader.steps('val'))):
        out = space_bert(batch['input_ids'].to(device)) # (B, 2)

        logits = out.logits.cpu()
        concept_spaces = [c.cpu() for c in out.concept_spaces]

        probs = F.softmax(logits, dim=-1).cpu()
        pred = torch.argmax(probs, dim=-1) # (B)
        val_preds += pred.detach().tolist()
        val_labels += [l.item() for l in batch['label_ids']]

        # CE + l1 * inter_loss + l2 * intra_loss
        loss = F.cross_entropy(logits, batch['label_ids']) + l1 * inter_space_loss(concept_spaces) + l2 * intra_space_loss(concept_spaces)

        val_loss += loss.item()

    print()
    print(f'Train loss: {train_loss / dataloader.steps("train")} | Val loss: {val_loss / dataloader.steps("val")}')
    print(f'Train acc: {accuracy_score(train_labels, train_preds)} | Val acc: {accuracy_score(val_labels, val_preds)}')
    print(f'Train f1: {f1_score(train_labels, train_preds)} | Val f1: {f1_score(val_labels, val_preds)}')

Epoch: 1


100%|██████████| 625/625 [00:05<00:00, 116.00it/s]
157it [00:00, 231.95it/s]                         



Train loss: 0.6956174991607667 | Val loss: 5567.979883438502
Train acc: 0.5101 | Val acc: 0.5054
Train f1: 0.5113216957605985 | Val f1: 0.6703985072637612
Epoch: 2


100%|██████████| 625/625 [00:03<00:00, 198.64it/s]
157it [00:00, 334.76it/s]                         



Train loss: 0.6931235491752624 | Val loss: 5530.198789938902
Train acc: 0.5141 | Val acc: 0.5046
Train f1: 0.5145854145854145 | Val f1: 0.6700412947915279
Epoch: 3


...


Train loss: 0.6923209614753724 | Val loss: 4875.984616010617
Train acc: 0.51565 | Val acc: 0.5046
Train f1: 0.5162546816479401 | Val f1: 0.6700412947915279
Epoch: 4


100%|██████████| 625/625 [00:02<00:00, 244.16it/s]
157it [00:00, 440.08it/s]                         



Train loss: 0.6919279090881347 | Val loss: 5133.811683067908
Train acc: 0.5174 | Val acc: 0.504
Train f1: 0.5151697809925658 | Val f1: 0.669773635153129
Epoch: 5


100%|██████████| 625/625 [00:02<00:00, 244.11it/s]
157it [00:00, 441.64it/s]                         


Train loss: 0.6911894980430603 | Val loss: 5574.917411608573
Train acc: 0.5207 | Val acc: 0.504
Train f1: 0.5222288676236044 | Val f1: 0.669773635153129


# Interpreting and visualizing the results

In [103]:
test_dataloader = IMDBDataloader(data, test_data, tokenizer, encode_label, batch_size=1)

Val Label Encoding: 100%|██████████| 5000/5000 [00:00<00:00, 749947.07it/s]


In [104]:
# examples with less than 16 words are easier to visualize, so focus on them
examples_ids = []
for i, v in enumerate(test_dataloader.splits['test']):
    if len(v) <= 16:
        examples_ids.append(i)
print(examples_ids)

[1959, 2939, 6394, 15789, 16349, 21487, 22019, 24588]


## Bert Embeddings

In [105]:
scatters = []
for sample_index in examples_ids:
    # extract example, decode to tokens and get the sequence length (ingoring padding)
    test_tokenized_batch = test_dataloader.peek_index_tokenized(index=sample_index, split='test')
    tokens = tokenizer.decode([t.item() for t in test_tokenized_batch['input_ids'][0] if t != 0], ignore_special=False).split(' ')[:MAX_SEQ_LEN]
    seq_len = len(tokens)

    embed = bert.nano_bert.embedding(test_tokenized_batch['input_ids'].to(device))

    x, y, z = embed[0, :seq_len, 0].detach().cpu().numpy(), embed[0, :seq_len, 1].detach().cpu().numpy(), embed[0, :seq_len, 2].detach().cpu().numpy()

    scatters.append(go.Scatter3d(
        x=x, y=y, z=z, mode='markers+text', name=f'Example: {sample_index}',
        text=tokens,
    ))

In [106]:
fig = go.Figure(
    data=scatters,
    layout=go.Layout(
        title=go.layout.Title(text='Embeddings')
    ))
fig.show()

In [107]:
scatters = []
for sample_index in examples_ids:
    # extract example, decode to tokens and get the sequence length (ingoring padding)
    test_tokenized_batch = test_dataloader.peek_index_tokenized(index=sample_index, split='test')
    tokens = tokenizer.decode([t.item() for t in test_tokenized_batch['input_ids'][0] if t != 0], ignore_special=False).split(' ')[:MAX_SEQ_LEN]
    seq_len = len(tokens)

    embed = bert.nano_bert(test_tokenized_batch['input_ids'].to(device))

    x, y, z = embed[0, :seq_len, 0].detach().cpu().numpy(), embed[0, :seq_len, 1].detach().cpu().numpy(), embed[0, :seq_len, 2].detach().cpu().numpy()

    scatters.append(go.Scatter3d(
        x=x, y=y, z=z, mode='markers+text', name=f'Example: {sample_index}',
        text=tokens,
    ))

In [108]:
fig = go.Figure(
    data=scatters,
    layout=go.Layout(
        title=go.layout.Title(text='Raw Embeddings')
    ))
fig.show()

## Space Embeddings

In [122]:
scatters = []
colors = ['blue', 'red']
for sample_index in examples_ids:
    # extract example, decode to tokens and get the sequence length (ingoring padding)
    test_tokenized_batch = test_dataloader.peek_index_tokenized(index=sample_index, split='test')
    tokens = tokenizer.decode([t.item() for t in test_tokenized_batch['input_ids'][0] if t != 0], ignore_special=False).split(' ')[:MAX_SEQ_LEN]
    seq_len = len(tokens)

    bert_embed = space_bert.bert(test_tokenized_batch['input_ids'].to(device))

    concept_spaces = space_bert.space_model.space_model(bert_embed).concept_spaces

    for c, embed in enumerate(concept_spaces):
        x, y, z = embed[0, :seq_len, 0].detach().cpu().numpy(), embed[0, :seq_len, 1].detach().cpu().numpy(), embed[0, :seq_len, 2].detach().cpu().numpy()

        scatters.append(go.Scatter3d(
            x=x, y=y, z=z, mode='markers+text',
            name=f'Example: {sample_index} ({c})',
            text=tokens,
            marker=dict(color=colors[c]),
            # hovertext=[]
        ))

In [123]:
fig = go.Figure(
    data=scatters,
    layout=go.Layout(
        title=go.layout.Title(text='Space Embeddings')
    ))
fig.show()